#### 定时调度器

---

##### sched

In [ ]:
import sched
import time
import random
from threading import Thread
from queue import Queue

scheduler = sched.scheduler(time.time, time.sleep)
goods = Queue()


def make():
    good = random.randrange(1, 100)
    goods.put(good)
    print(f"Produced: {good}")


def procude() -> None:
    while True:
        scheduler.enter(1, 1, make)
        scheduler.run()


def consume() -> None:
    while True:
        time.sleep(random.randint(1, 3))
        if goods.empty():
            print(f"Nothing to consume...")
            continue
        good = goods.get()
        print(f"Consumed: {good}")


producer = Thread(target=procude)
producer.start()
consumer = Thread(target=consume)
consumer.start()

##### APScheduler

- 基本调度器设置

In [5]:
from apscheduler.schedulers.background import BackgroundScheduler

scheduler = BackgroundScheduler()
scheduler.configure(timezone='UTC')  # 设置调度器参数

- 添加/管理任务

In [6]:
def job():
    print("任务执行")

# 添加任务
scheduler.add_job(job, 'interval', seconds=2, id='my_job')

# 获取任务
job_ref = scheduler.get_job('my_job')

# 修改任务
scheduler.modify_job('my_job', seconds=5)

# 暂停任务
scheduler.pause_job('my_job')

# 恢复任务
scheduler.resume_job('my_job')

# 删除任务
scheduler.remove_job('my_job')

# 删除所有任务
scheduler.remove_all_jobs()

# 查看所有任务
scheduler.print_jobs()

AttributeError: The following are not modifiable attributes of Job: seconds

- 添加/删除执行器（executor）

In [ ]:
from apscheduler.executors.pool import ThreadPoolExecutor

# 添加执行器（最多3个并发线程）
scheduler.add_executor(ThreadPoolExecutor(3), alias='my_executor')

# 移除执行器
scheduler.remove_executor('my_executor')

- 添加/删除任务存储（jobstore）

In [ ]:
from apscheduler.jobstores.memory import MemoryJobStore

# 添加任务存储
scheduler.add_jobstore(MemoryJobStore(), alias='my_store')

# 删除任务存储
scheduler.remove_jobstore('my_store')

- 添加/移除监听器 (listener)

In [ ]:
from apscheduler.events import EVENT_JOB_EXECUTED, EVENT_JOB_ERROR

def listener(event):
    if event.exception:
        print("任务出错")
    else:
        print("任务成功")

# 添加监听器
scheduler.add_listener(listener, EVENT_JOB_EXECUTED | EVENT_JOB_ERROR)

# 移除监听器
scheduler.remove_listener(listener)

- 控制调度器运行状态

In [ ]:
# 启动
scheduler.start()

# 暂停整个调度器
scheduler.pause()

# 恢复整个调度器
scheduler.resume()

# 唤醒调度器（立即处理下一个任务）
scheduler.wakeup()

# 停止调度器
scheduler.shutdown(wait=True)

- 导入/导出任务（依赖 jobstore 支持）

In [ ]:
# 导出任务
jobs = scheduler.export_jobs()

# 导入任务
scheduler.import_jobs(jobs)

##### 两者区别

- sched 不支持周期循环
- APScheduler 更加稳定